In [0]:
from tweepy import API
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream


import json
import sys
import pandas as pd

In [0]:
ACCESS_TOKEN = '--'
ACCESS_TOKEN_SECRET = '--'
CONSUMER_KEY = '--'
CONSUMER_SECRET = '--'

In [0]:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

In [0]:
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

class Listener(StreamListener):
    def __init__(self):
        super(Listener,self).__init__()
        self.output_list = [ ]
    def on_status(self, status):
        self.output_list.append(status)

    def on_error(self, status_code):
        print(status_code)
        return False


In [0]:
#output = open('stream_output.txt', 'w')

l = Listener()


In [0]:
stream = Stream(auth=api.auth, listener=l)

try:
    print('Start streaming.')
    stream.sample(languages=['en'])
    
    
    #tracking target  SF - LA 34.059990, -118.372343
    stream.filter(locations = [-122.75,36.8,-121.75,37.8,  -118.5,34,-117,33],
                  track = ['movies', 'netflix', 'movies to watch'],
                  is_async=True)
    
except KeyboardInterrupt:
    print("Stopped.")
finally:
    print('Done.')
    stream.disconnect()

Start streaming.
Stopped.
Done.


In [0]:
len(l.output_list)

26487

In [0]:
#convert jon to dataframe

dict_ = {'screen_name': [], 'id': [], 'location': [], 'created_at': [], 'text': [], 'favorite_count': [],
        'description': [], 'profile_background_color': [], 'profile_link_color': [],
        'profile_sidebar_border_color': [], 'profile_sidebar_fill_color': [], 'profile_text_color': []}


for i in range(0,len(l.output_list)):
    dict_['screen_name'].append(l.output_list[i]._json['user']['screen_name'])
    dict_['id'].append(l.output_list[i]._json['user']['id'])
    dict_['location'].append(l.output_list[i]._json['user']['location'])
    dict_['description'].append(l.output_list[i]._json['user']['description'])
    dict_['profile_background_color'].append(l.output_list[i]._json['user']['profile_background_color'])
    dict_['profile_link_color'].append(l.output_list[i]._json['user']['profile_link_color'])
    dict_['profile_sidebar_border_color'].append(l.output_list[i]._json['user']['profile_sidebar_border_color'])
    dict_['profile_sidebar_fill_color'].append(l.output_list[i]._json['user']['profile_sidebar_fill_color'])
    dict_['profile_text_color'].append(l.output_list[i]._json['user']['profile_text_color'])
    
    
    dict_['created_at'].append(l.output_list[i]._json['created_at'])
    dict_['text'].append(l.output_list[i]._json['text'])
    dict_['favorite_count'].append(l.output_list[i]._json['favorite_count'])


In [0]:
df = pd.DataFrame(dict_)
df.sort_values(by='created_at', inplace=True, ascending=False)

In [0]:
pd.set_option('display.max_colwidth', -1)
print(df['text'])

26486    As if the Government didn't know that!\n\nWhy do Courts have to meddle in everything? If this isn't overreach, what i… https://t.co/7y4r9JNk6l
26480    RT @gooninggayperv: Serve all penis https://t.co/FRtCaOKbAO                                                                                   
26474    Not finna be on here arguing with tar roach looking ass bitches 😂😂😂                                                                           
26475    @VANDERW00D i hate u                                                                                                                          
26476    shame on you                                                                                                                                  
             ...                                                                                                                                       
6        RT @LouieDi13: What does Prime Liverpool mean lol! He destroyed all teams at th

In [0]:
df['location'].str.contains('CA').sum()

604

In [0]:
df['location'].isna().sum()

9137

In [0]:
df.to_csv('tweets2.csv')